# Interroger des documents (RAG)

In [5]:
%pip install -qU wget openai


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


Commencez par télécharger le document qui va vous servir de base de connaissance.

In [6]:
# Download a file
import os

import wget

file_path = "my_document.pdf"
if os.path.exists(file_path):
    os.remove(file_path)
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

'my_document.pdf'

Puis instancier la connexion à l'API Albert.

In [7]:
# OpenAI client configuration
import requests
from openai import OpenAI
import os

print(os.getenv("API_KEY"))

base_url = "http://localhost:8000/v1"
api_key = os.getenv("API_KEY")

client = OpenAI(base_url=base_url, api_key=api_key)

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}

anthony-eZ8Sagt5sFyDEk5EaRd6KA6635492Rp9dtc3isqzKS6A7528EZg4Yp54eWVD9KbX


Vous aurez besoin pour la suite d'un modèle de langage ainsi qu'un modèle d'embeddings (pour le RAG). Pour cela vous pouvez appelez le endpoint `/v1/models` pour obtenir la liste des modèles. Les modèles de langage ont le type *text-generation* et les modèles d'embeddings le type *text-embeddings-inference*.

In [8]:
language_model, embeddings_model = None, None

for model in client.models.list().data:
    if model.type == "text-generation" and language_model is None:
        language_model = model.id
    if model.type == "text-embeddings-inference" and embeddings_model is None:
        embeddings_model = model.id

print(f"language model: {language_model}\nembeddings model: {embeddings_model}")

language model: meta-llama/Meta-Llama-3.1-70B-Instruct
embeddings model: intfloat/multilingual-e5-large


Enfin pour vous importer le document dans une collection de notre base vectorielle à l'aide du endpoint POST `/v1/files`.

Vous devez spécifier le modèle d'embeddings qui sera utilisé pour vectoriser votre document. Vous pouvez trouver la liste des modèles avec le endpoint `/v1/models`. Les modèles d'embeddings sont indiqués avec le type _feature-extraction_.

Le endpoint POST `/v1/files` doit retourner un status _success_.

In [9]:
# Remove previous files
collection = "tutorial" # "internet" is reserved for the internet search tool
response = session.delete(f"{base_url}/files/{collection}")
response.status_code

204

In [10]:
# Upload a file
model = "intfloat/multilingual-e5-large"
params = {"collection": collection, "embeddings_model": embeddings_model}

files = {"files": (os.path.basename(file_path), open(file_path, "rb"), "application/pdf")}
response = session.post(f"{base_url}/files", params=params, files=files)

response.json()

{'object': 'list',
 'data': [{'object': 'upload',
   'id': 'e5da4a7f-938a-49a3-8fb7-c32fca4f52d2',
   'filename': 'my_document.pdf',
   'status': 'success'}]}

Vous pouvez observer les fichiers que vous avez importer dans une collection à l'aide du endpoint GET `/v1/files.`

In [11]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")
response.json()
file_id = response.json()["data"][0]["id"]
print(file_id)

e5da4a7f-938a-49a3-8fb7-c32fca4f52d2


In [12]:
# Display tools parameters
response = session.get(f"{base_url}/tools")
for tool in response.json()["data"]:
    if tool["id"] == "BaseRAG":
        print(tool["description"].strip())

Base RAG, basic retrival augmented generation.

    Args:
        embeddings_model (str): OpenAI embeddings model
        collection (List[str], optional): List of collections to search in. Defaults to None (all collections).
        file_ids (Optional[List[str]], optional): List of file IDs in the selected collections (after upload files). Defaults to None (all files are selected).
        k (int, optional): Top K per collection. Defaults to 4.
        prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.

    DEFAULT_PROMPT_TEMPLATE:
        "Réponds à la question suivante en te basant sur les documents ci-dessous : {prompt}

Documents :

{documents}"


In [15]:
# Chat completions
data = {
    "model": language_model,
    "messages": [{"role": "user", "content": "Qui est Ulrich Tan ?"}],
    "stream": False,
    "n": 1,
    "tools": [
        {
            "function": {
                "name": "BaseRAG",
                "parameters": {
                    "embeddings_model": embeddings_model,
                    "collections": [collection, "internet"],
                    "k": 2,
                },
            },
            "type": "function",
        }
    ],
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

APIConnectionError: Connection error.

Vous pouvez récupérer les metadata des tools utilisées. Ces metadata vous donnera, entre autre, le prompt envoyé au modèle.

In [ ]:
print(response.metadata[0]["BaseRAG"]["prompt"])

Réponds à la question suivante en te basant sur les documents ci-dessous : Qui est Ulrich Tan ?

Documents :

–  M. Gilles Corbi, agent contractuel, chef du pôle Production du département « ISO » ; –  M. Yann Brûlé, grade, chef du pôle « SI RIE » du département « ISO » ; –  M. Ulrich Tan, chef du pôle Datamin du département « Etalab » ; –  Mme Marie-Christie Ritz, cheffe du pôle « RH et attractivité » ; –  Mme Fadila Leturcq, cheffe du pôle « Campus du numérique ». 21  juin 2023 JOURNAL  OFFICIEL  DE  LA  RÉPUBLIQUE  FRANÇAISE Texte 5 sur 95

–  M.  Louis  di  Benedetto,  inspecteur  en  chef  de  la  santé  publique  vétérinaire,  chef  du  pôle  « Pilotage  de  la qualité et des partenariats » du département « ISO » ; –  M.  Philippe  Levillain,  agent  contractuel,  chef  du  pôle  « Réseau  interministériel  de  l’Etat  /  SOI »  du département « ISO » ;

Le quadra, à la tête du DataLab, a créé Albert, l’IA générative souveraine à la française, qui permet à l’État d’être plus rapid